In [69]:
!pip install keras-tuner

In [70]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [71]:
fashion_mnist=keras.datasets.fashion_mnist

In [72]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [73]:
train_images=train_images/255.0
train_images=test_images/255.0

In [74]:
train_images[0].shape

(28, 28)

In [75]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [76]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]), 
        activation='relu',
        input_shape=(28,28,1)
    ),
     keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16), 
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])
  return model

In [77]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [78]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output', project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fashion/tuner0.json


In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 3 Complete [00h 04m 22s]
val_accuracy: 0.0949999988079071

Best val_accuracy So Far: 0.10300000011920929
Total elapsed time: 00h 12m 08s

Search: Running Trial #4

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |128               |96                
conv_1_kernel     |5                 |3                 
conv_2_filter     |32                |48                
conv_2_kernel     |5                 |5                 
dense_1_units     |96                |64                
learning_rate     |0.01              |0.01              

Epoch 1/3
282/282 [==============================] - 98s 346ms/step - loss: 2.4310 - accuracy: 0.0962 - val_loss: 2.3065 - val_accuracy: 0.0900
Epoch 2/3
282/282 [==============================] - 92s 326ms/step - loss: 2.3041 - accuracy: 0.1026 - val_loss: 2.3036 - val_accuracy: 0.1020
Epoch 3/3
152/282 [===============>..............] - ETA: 40s - loss: 2.3039 - accuracy: 0.1034

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()